# Учебный Проект → «Викишоп» 📚

>Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Импортируем библиотеки 🎒

In [1]:
import re
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import torch
# import torch.nn as nn
# import torchtext

from tqdm.notebook import tqdm
from tqdm.notebook import trange

from numpy.random import RandomState

from matplotlib import rcParams

from pytorch_pretrained_bert import BertTokenizer
from pytorch_pretrained_bert import BertConfig
from pytorch_pretrained_bert import BertModel

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from catboost import CatBoostRegressor

# Импорты роняют Kernel
# The kernel for 13_project/13_project.ipynb appears to have died. It will restart automatically.
# from transformers import BertConfig
# from transformers import BertModel

# import tensorflow as tf
# import tensorflow_hub as tf_hub
# import tensorflow_text as tf_text

In [2]:
# set max columns to none
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)

Константы.

In [3]:
RANDOM_SEED = 108108108
RANDOM_NP = RandomState(128)

Функции.

<hr>

## Загрузим данные

Столбец `text` в нём содержит текст комментария, а `toxic` — целевой признак.

In [4]:
try:
    comments_df = pd.read_csv(
        './datasets/toxic_comments.csv',
        usecols=['text', 'toxic']
    )
    
except FileNotFoundError:
    comments_df = pd.read_csv(
        'https://code.s3.yandex.net/datasets/toxic_comments.csv',
        usecols=['text', 'toxic']
    )
    print('FYI datasets loaded via url')

In [5]:
display(
    comments_df.head(2),
    comments_df.sample(2),
    comments_df.tail(2),
    comments_df['toxic'].describe(),
)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0


,text,toxic
90807,Hello . I listed the list of all parts and sec...,0
92476,", and I changed it to state thus so.",0


,text,toxic
159290,And it looks like it was actually you who put ...,0
159291,"""\nAnd ... I really don't think you understand...",0


count    159292.000000
mean          0.101612
std           0.302139
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: toxic, dtype: float64

In [6]:
print(
    'Токсичных комментариев: {:.2%}'.format(
            len(comments_df.loc[comments_df['toxic'] == 1].index)
            / len(comments_df.index)
        )
)

Токсичных комментариев: 10.16%


Дисбаланс, однако.

Хотел дополнить, но похоже нашел тот же набор данных на kaggle

## Подготовим данные

In [7]:
comments_df['text'] = comments_df['text'].str.lower()

In [8]:
re_url = re.compile(r'http\S+')

In [9]:
re_email = re.compile(r'\S*@\S*\s?')

In [10]:
re_symbols = re.compile('[(){}\[\]"\'~:;><»=|—+-_#0123456789!&?$%^*]')

In [11]:
re_new_line = re.compile('(\n)+')

In [12]:
re_space = re.compile(r'(\s)+')

In [13]:
comments_df['text'] = comments_df['text'].apply(
        lambda twt: re.sub(re_url, '', twt)
    )

In [14]:
comments_df['text'] = comments_df['text'].apply(
        lambda twt: re.sub(re_email, '', twt)
    )

In [15]:
comments_df['text'] = comments_df['text'].apply(
        lambda twt: re_symbols.sub(' ', twt)
    )

In [16]:
comments_df['text'] = comments_df['text'].apply(
        lambda twt: re.sub(re_new_line, ' ', twt)
    )

In [17]:
comments_df['text'] = comments_df['text'].apply(
        lambda twt: re.sub(re_space, ' ', twt)
    )

In [18]:
comments_df.sample(5)

,text,toxic
78977,sorry i meant to put that last edit in the tal...,0
156492,stop consensus the consensus is incorrect my e...,0
126591,what size are the targets and what is the accu...,0
5672,response hi checking in this is a response i w...,0
27917,george r vito accomplished senior management p...,0


Загрузите и подготовьте данные.
Обучите разные модели.
Сделайте выводы.

Зададим `BertTokenizer`

In [19]:
tokenizer = BertTokenizer.from_pretrained(
        'bert-base-uncased',
        # do_basic_tokenize=False,
    )

Проверим `BertTokenizer`

In [20]:
text = comments_df['text'].sample(1).values[0]
tokenized_text = tokenizer.tokenize(text)

print(tokenized_text)
print(tokenizer.convert_tokens_to_ids(tokenized_text))

['yes', 'see', 'new', 'reference']
[2748, 2156, 2047, 4431]


In [21]:
%%time

comments_df['text'] = comments_df['text'].apply(
        lambda twt:  tokenizer.convert_tokens_to_ids(
            tokenizer.tokenize(twt)
        )
    )

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (540 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (561 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (516 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1132 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (686 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length 

CPU times: user 5min 19s, sys: 1.46 s, total: 5min 21s
Wall time: 5min 22s


In [22]:
comments_df.drop(
    index=comments_df.loc[
            comments_df['text'].map(len) > 512
        ].index,
    inplace=True,
)

Только чтобы не ждать 83 часа, сделаем downsampling.  
То есть это только в учебных целях и для экономии времени.

In [23]:
idxs_toxic = comments_df.loc[comments_df['toxic'] == 1].index
idxs_non_toxic = comments_df.loc[comments_df['toxic'] == 0].index

index_to_drop = pd.Index(
        np.random.choice(
            idxs_non_toxic.to_numpy(),
            size=len(idxs_non_toxic) - len(idxs_toxic),
            replace=False,
        )
    )

In [24]:
comments_df.drop(index=index_to_drop, inplace=True)

In [28]:
features = comments_df['text'].to_numpy(dtype='int16')
targets = comments_df['toxic'].to_numpy(dtype='int16')

NameError: name 'comments_df' is not defined

In [26]:
del comments_df
del idxs_toxic
del idxs_non_toxic
del index_to_drop
del tokenizer

In [27]:
display(features.shape)

(31814,)

In [32]:
%%time

max_len = 0
for i in features:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in features])

attention_mask = np.where(padded != 0, 1, 0)

CPU times: user 2.65 s, sys: 16.2 ms, total: 2.67 s
Wall time: 2.66 s


In [33]:
config = BertConfig.from_json_file(
    './bert/bert-base-uncased-config.json')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
batch_size = 100
embeddings = []
seq = 0
for i in trange(padded.shape[0] // batch_size):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)[0]
        # embeddings.append(batch_embeddings[0][:,0,:].numpy())
        batch_embeddings[0][:,0,:].numpy().tofile(f'./embeddings/emb_{seq}')
        seq += 1

  0%|          | 0/318 [00:00<?, ?it/s]

In [201]:
features = np.concatenate(embeddings)

>Все ли шаги по инструкции выполняете?  
Как готовите данные?  
Как анализируете данные?  
Какие модели и гиперпараметры рассматриваете?  
Не дублируете ли код?  
Какие выводы делаете?  
Следите ли за структурой проекта?  
Поддерживаете аккуратность кода?  

In [ ]:
PRETRAINED_VOCAB_FILES_MAP = {
    'vocab_file':
    {
        'bert-base-uncased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt",
        'bert-large-uncased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-vocab.txt",
        'bert-base-cased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt",
        'bert-large-cased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-vocab.txt",
        'bert-base-multilingual-uncased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-vocab.txt",
        'bert-base-multilingual-cased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt",
        'bert-base-chinese': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt",
        'bert-base-german-cased': "https://int-deepset-models-bert.s3.eu-central-1.amazonaws.com/pytorch/bert-base-german-cased-vocab.txt",
        'bert-large-uncased-whole-word-masking': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-vocab.txt",
        'bert-large-cased-whole-word-masking': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-whole-word-masking-vocab.txt",
        'bert-large-uncased-whole-word-masking-finetuned-squad': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-vocab.txt",
        'bert-large-cased-whole-word-masking-finetuned-squad': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-whole-word-masking-finetuned-squad-vocab.txt",
        'bert-base-cased-finetuned-mrpc': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-finetuned-mrpc-vocab.txt",
    }
}

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('russian'))

data = pd.read_csv("/datasets/tweets_lemm.csv")
corpus = data['lemm_text'].values.astype('U')

# создайте мешок слов без учёта стоп-слов
count_vect = CountVectorizer()
bow = count_vect.fit_transform(corpus)

print("Размер мешка без учёта стоп-слов:", bow.shape)

# создайте новый мешок слов с учётом стоп-слов
count_vect = CountVectorizer(stop_words=stop_words)
bow = count_vect.fit_transform(corpus)

print("Размер мешка с учётом стоп-слов:", bow.shape)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

data = pd.read_csv("/datasets/tweets_lemm.csv")
corpus = list(data['lemm_text'])

# создайте n-грамму n_gramm, для которой n=2
count_vect = CountVectorizer(ngram_range=(2, 2))
n_gramm = count_vect.fit_transform(corpus)
print("Размер:", n_gramm.shape)

$$ \large
TF = \dfrac{t}{n}
$$

>где t (от англ. term) — количество употребления слова, а n — общее число слов в тексте.  
IDF нужна в формуле, чтобы уменьшить вес слов, наиболее распространённых в любом другом тексте заданного корпуса. IDF зависит от общего числа текстов в корпусе (D) и количества текстов, в которых это слово встречается (d). 

$$ \large
IDF = \log_{10}\dfrac{D}{d}
$$

$$ \large
TFIDF = TF * IDF
$$

> Большая величина TF-IDF говорит об уникальности слова в тексте по отношению к корпусу.  
Чем чаще оно встречается в конкретном тексте и реже в остальных, тем выше значение TF-IDF.

In [ ]:
text = [
    'Карл у Клара украсть коралл',
    'Клара у Карл украсть кларнет',
    'Клара строго карать Карл за кража коралл'
]

In [ ]:
tf = 1/(len(text[2].split()) - 1)
print(tf)

In [ ]:
poem = []
poem.extend(text[0].split())
poem.extend(text[1].split())
poem.extend(text[2].split())
display(set(poem))
display(len(set(poem)))

In [ ]:
import math
 
tf = 1 / len(text[2].split())

idf = math.log(3/2,10)
print (idf)
tfidf = tf * idf
print (tfidf)

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

data = pd.read_csv("/datasets/tweets_lemm.csv")
corpus = data['lemm_text'].values.astype('U')

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('russian'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit_transform(corpus)

print("Размер матрицы:", tf_idf.shape)

In [ ]:
tweets_lemm_train = pd.read_csv(
        './datasets/tweets_lemm_train.csv',
    )

tweets_lemm_test= pd.read_csv(
        './datasets/tweets_lemm_test.csv',
    )